# Using Huggingface Transformers with Tune

# Required Packages and Variables

In [ ]:
!pip install -q transformers[torch] "ray[tune]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 70.6 MB/s eta 0:00:00


In [ ]:
"""
This example is uses the official
huggingface transformers `hyperparameter_search` API.
"""
import os
import pandas as pd

import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.examples.pbt_transformers.utils import (
    download_data,
    build_compute_metrics_fn,
)
from ray.tune.schedulers import PopulationBasedTraining
from transformers import (
    glue_tasks_num_labels,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    GlueDataset,
    GlueDataTrainingArguments,
    TrainingArguments,
)

In [ ]:
# glue_tasks_num_labels = {
    # "cola": 2,        # CoLA evaluates the grammaticality of sentences by determining whether they are linguistically acceptable or not.
    #                     Example: Sentence: “She eats pizza.” Label: Acceptable ||| Sentence: “Eats she pizza.” Label: Unacceptable
    #                     Use Case: CoLA helps LLMs understand syntactic structures and grammatical correctness.
    # "mnli": 3,        # MNLI assesses textual entailment, determining whether a hypothesis can be inferred from a given premise.
    #                     Example: Premise: “The cat chased the mouse.” Hypothesis: “The mouse was caught.” Label: Entailment
    #                     Use Case: MNLI is crucial for applications like question answering and information retrieval.
    # "mrpc": 2,        # MRPC focuses on paraphrase identification, distinguishing between sentences with similar meanings.
    #                     Example: Sentence 1: “The sun rises in the east.” Sentence 2: “The east sees the sun rise.” Label: Paraphrase
    #                     Use Case: MRPC aids LLMs in understanding semantic equivalence.
    # "sst-2": 2,       # SST-2 classifies sentences as positive or negative sentiment.
    #                     Example: Sentence: “The movie was fantastic!” Label: Positive
    #                     Use Case: SST-2 enables sentiment analysis in various applications.
    # "sts-b": 1,       # STS-B quantifies the semantic similarity between sentence pairs.
    #                     Example: Sentence 1: “The cat is on the mat.” Sentence 2: “The mat holds the cat.” Similarity Score: 4.5 (on a scale of 1 to 5)
    #                     Use Case: STS-B aids in measuring semantic relatedness.
    # "qqp": 2,         # QQP identifies duplicate question pairs.
    #                     Example: Question 1: “How does photosynthesis work?” Question 2: “What is the process of photosynthesis?” Label: Duplicate
    #                     Use Case: QQP assists in question matching and duplicate detection.
    # "qnli": 2,        # QNLI is similar to MNLI but focuses on question answering.
    #                     Example: Premise: “The cat chased the mouse.” Hypothesis: “Did the cat catch the mouse?” Label: Entailment
    #                     Use Case: QNLI supports question-answering systems.
    # "rte": 2,         # RTE determines whether a hypothesis logically follows from a given premise.
    #                     Example: Premise: “The cat chased the mouse.” Hypothesis: “The mouse escaped.” (entailment).
    #                     Use Case: RTE is essential for understanding logical relationships.
    # "wnli": 2,        # WNLI involves resolving pronoun references in ambiguous sentences.
    #                     Example: Sentence: “The cat chased the dog. It was fast.” Label: Entailment
    #                     Use Case: WNLI helps LLMs handle coreference resolution.
    # }

In [ ]:
smoke_test = False       # if True: use dummy data
num_samples=8           # number of trials
gpus_per_trial=1

# Data Preparing

In [ ]:
data_dir_name = "./data" if not smoke_test else "./test_data"
data_dir = os.path.abspath(os.path.join(os.getcwd(), data_dir_name))
if not os.path.exists(data_dir):
    os.mkdir(data_dir, 0o755)

task_name = "rte"
task_data_dir = os.path.join(data_dir, task_name.upper())
num_labels = glue_tasks_num_labels[task_name]                 # number of output classes(labels)

# Tokenizer & Model With Config

In [ ]:
model_name = (
    "bert-base-uncased" if not smoke_test else "sshleifer/tiny-distilroberta-base"
)

config = AutoConfig.from_pretrained(                              # from transformers
    model_name, num_labels=num_labels, finetuning_task=task_name
)

# Download and cache tokenizer, model, and features
print("Downloading and caching Tokenizer")
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Triggers tokenizer download to cache
print("Downloading and caching pre-trained model")
AutoModelForSequenceClassification.from_pretrained(
    model_name,
    config=config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
def get_model():
    return AutoModelForSequenceClassification.from_pretrained(
        model_name,
        config=config,
    )

# Dataset (GLUE)

In [ ]:
# Download data.
download_data(task_name, data_dir)

data_args = GlueDataTrainingArguments(task_name=task_name, data_dir=task_data_dir)

train_dataset = GlueDataset(
    data_args, tokenizer=tokenizer, mode="train", cache_dir=task_data_dir
)
eval_dataset = GlueDataset(
    data_args, tokenizer=tokenizer, mode="dev", cache_dir=task_data_dir
)


Data already exists. Using downloaded data for task rte from /content/data


# Training

In [ ]:
training_args = TrainingArguments(
    output_dir=".",
    learning_rate=1e-5,  # config
    do_train=True,
    do_eval=True,
    no_cuda=gpus_per_trial <= 0,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    num_train_epochs=2,  # config
    max_steps=-1,
    per_device_train_batch_size=16,  # config
    per_device_eval_batch_size=16,  # config
    warmup_steps=0,
    weight_decay=0.1,  # config
    logging_dir="./logs",
    skip_memory_metrics=True,
    report_to="none",
)
# Defining the training function
trainer = Trainer(
    model_init=get_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=build_compute_metrics_fn(task_name),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Hyperparameter Tuning

In [ ]:
# Defining the search space
tune_config = {
    "per_device_train_batch_size": 32,
    "per_device_eval_batch_size": 32,
    "num_train_epochs": tune.choice([2, 3, 4, 5]),
    "max_steps": 1 if smoke_test else -1,  # Used for smoke test.
}

# Defining the scheduler
scheduler = PopulationBasedTraining(
    time_attr="training_iteration",
    metric="eval_acc",
    mode="max",
    perturbation_interval=1,
    hyperparam_mutations={
        "weight_decay": tune.uniform(0.0, 0.3),
        "learning_rate": tune.uniform(1e-5, 5e-5),
        "per_device_train_batch_size": [16, 32, 64],
    },
)

# Defining the reporter
# Report table tp print in output
reporter = CLIReporter(
    parameter_columns={
        "weight_decay": "w_decay",
        "learning_rate": "lr",
        "per_device_train_batch_size": "train_bs/gpu",
        "num_train_epochs": "num_epochs",
    },
    metric_columns=["eval_acc", "eval_loss", "epoch", "training_iteration"],
)

# Starting the distributed hyperparameter tuning process
trainer.hyperparameter_search(
    hp_space=lambda _: tune_config,
    backend="ray",
    n_trials=num_samples,
    resources_per_trial={"cpu": 1, "gpu": gpus_per_trial},
    scheduler=scheduler,
    keep_checkpoints_num=1,
    checkpoint_score_attr="training_iteration",
    stop={"training_iteration": 1} if smoke_test else None,
    progress_reporter=reporter,
    storage_path="~/ray_results/",
    name="tune_transformer_pbt",
    log_to_file=True,
)

2024-06-09 14:18:05,237	WARNING tune.py:902 -- AIR_VERBOSITY is set, ignoring passed-in ProgressReporter for now.
/usr/local/lib/python3.10/dist-packages/ray/tune/schedulers/pbt.py:480: UserWarning: Using `CheckpointConfig.num_to_keep <= 2` with PBT can lead to restoration problems when checkpoint are deleted too early for other trials to exploit them. If this happens, increase the value of `num_to_keep`.
  warnings.warn(


+----------------------------------------------------------+
| Configuration for experiment     tune_transformer_pbt    |
+----------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator   |
| Scheduler                        PopulationBasedTraining |
| Number of trials                 8                       |
+----------------------------------------------------------+

View detailed results here: /root/ray_results/tune_transformer_pbt
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-06-09_14-16-45_081420_699/artifacts/2024-06-09_14-18-05/tune_transformer_pbt/driver_artifacts`

Trial status: 8 PENDING
Current time: 2024-06-09 14:18:05. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:T4)
+--------------------------------------------------------+
| Trial name               status       num_train_epochs |
+------------------------------------------

(_objective pid=5968) 2024-06-09 14:18:18.280726: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(_objective pid=5968) 2024-06-09 14:18:18.280795: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(_objective pid=5968) 2024-06-09 14:18:18.287489: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(_objective pid=5968) 2024-06-09 14:18:20.650651: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(_objective pid=5968) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly ini


Trial status: 1 RUNNING | 7 PENDING
Current time: 2024-06-09 14:18:35. Total running time: 30s
Logical resource usage: 1.0/2 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
+--------------------------------------------------------+
| Trial name               status       num_train_epochs |
+--------------------------------------------------------+
| _objective_165d9_00000   RUNNING                     4 |
| _objective_165d9_00001   PENDING                     5 |
| _objective_165d9_00002   PENDING                     4 |
| _objective_165d9_00003   PENDING                     2 |
| _objective_165d9_00004   PENDING                     3 |
| _objective_165d9_00005   PENDING                     3 |
| _objective_165d9_00006   PENDING                     2 |
| _objective_165d9_00007   PENDING                     2 |
+--------------------------------------------------------+


 20%|█▉        | 62/312 [00:40<02:44,  1.52it/s]


...

2024-06-09 15:03:49,383	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/tune_transformer_pbt' in 0.0097s.



Trial _objective_165d9_00002 completed after 4 iterations at 2024-06-09 15:03:49. Total running time: 45min 44s

Trial status: 6 TERMINATED | 2 ERROR
Current time: 2024-06-09 15:03:49. Total running time: 45min 44s
Logical resource usage: 1.0/2 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name               status         num_train_epochs     iter     total time (s)     eval_loss     eval_acc     eval_runtime     ...amples_per_second |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
| _objective_165d9_00000   TERMINATED                    4        4            391.03       0.745496     0.6787             2.028                   136.585 |
| _objective_165d9_00001   TERMINATED                    4        4        

TuneError: ('Trials did not complete', [_objective_165d9_00005, _objective_165d9_00007])

(_objective pid=18837) {'train_runtime': 0.0043, 'train_samples_per_second': 2308791.193, 'train_steps_per_second': 72323.58, 'train_loss': 0.0, 'epoch': 4.0}


  0%|          | 0/312 [00:00<?, ?it/s]


# pbt_transformers.utils functions

In [ ]:
"""Utilities to load and cache data."""

import os
from typing import Callable, Dict
import numpy as np
from transformers import EvalPrediction
from transformers import glue_compute_metrics, glue_output_modes


def build_compute_metrics_fn(task_name: str) -> Callable[[EvalPrediction], Dict]:
    """Function from transformers/examples/text-classification/run_glue.py"""
    output_mode = glue_output_modes[task_name]

    def compute_metrics_fn(p: EvalPrediction):
        if output_mode == "classification":
            preds = np.argmax(p.predictions, axis=1)
        elif output_mode == "regression":
            preds = np.squeeze(p.predictions)
        metrics = glue_compute_metrics(task_name, preds, p.label_ids)
        return metrics

    return compute_metrics_fn

In [ ]:
def download_data(task_name, data_dir="./data"):
    # Download RTE training data
    print("Downloading dataset.")
    import urllib
    import zipfile

    if task_name == "rte":
        url = "https://dl.fbaipublicfiles.com/glue/data/RTE.zip"
    else:
        raise ValueError("Unknown task: {}".format(task_name))
    data_file = os.path.join(data_dir, "{}.zip".format(task_name))
    if not os.path.exists(data_file):
        urllib.request.urlretrieve(url, data_file)
        with zipfile.ZipFile(data_file) as zip_ref:
            zip_ref.extractall(data_dir)
        print("Downloaded data for task {} to {}".format(task_name, data_dir))
    else:
        print(
            "Data already exists. Using downloaded data for task {} from {}".format(
                task_name, data_dir
            )
        )